In [1]:
# SETUP THE MONGO_CONNECTION
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
#!pip install ollama
import ollama
ollama.pull('llama3.1')
uri = "mongodb://localhost:27017"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [2]:
db = client["linkedin-scrap"]
posts_collection = db["posts"]


In [13]:
pending_posts = list(posts_collection.find({"status":"unqualified"}))

In [14]:
PROMT1 = "You are my social media manager. I will provide you with a post, and you must tell me if it aligns with my interests. Response: You can only reply with either 'yes' or 'no'. Do not include any additional words or sentences. My Interest: I am seeking new opportunities in software development roles, having graduated in 2024 with around 1 year of experience as a full-stack developer (front-end and back-end). I am also interested in roles such as software engineer, web developer, backend engineer, frontend developer, and full-stack engineer. Posts mentioning hiring opportunities, offering referrals, or related to these roles are of the highest interest to me."
PROMT2 = "You are my social media manager , i will give you a post and you have to tell me if that post is good for my interest or not. RESPONSE: you can only respond in one word with either 'yes' or 'no', DO NOT ADD ANY OTHER SENTENCE TO IT,  MY INTEREST : I am looking for new opportunites in the software developer and realated  roles,  i passed out in 2024 and i have around 1 year of experience , I am working as a full stack developer and can work on both frontends and back ends, any post which mentions an hiring opportunity or providing reffral are my of AT MOST INTREST"
def isRelevant( query):
    if not query :
        return False;
    stream = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content': PROMT2
        },
        {
            'role': 'user',
            'content': "This is this post" + query,
        }]
    )
    stream  = stream['message']['content']
    response =""
    isQualified = stream.lower().find("yes") != -1
    print("BOT: ", stream)
    return isQualified;
    


    # print( "\n Chat line ", chatLine);
    # print( "\n Chat History", chatHistory)

In [15]:
for post in pending_posts:
    isQualified = isRelevant(post["feed_text"])
    if isQualified :
        post["status"] = "qualified"
        print(post["feed_text"])
    else :
        post["status"] = "unqualified"
    posts_collection.replace_one({"_id": post["_id"]}, post)

BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  no
BOT:  no
BOT:  No
BOT:  no
BOT:  no
BOT:  no
BOT:  no
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  no
BOT:  No
BOT:  No
BOT:  no
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  no
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  no
BOT:  no
BOT:  No
BOT:  No
BOT:  No
BOT:  No
BOT:  no
BOT:  no
BOT:  no
BOT:  no
BOT:  No
BOT:  no
BOT:  No
BOT:  No
BOT:  No
BOT:  Yes

 Last year my DMs were flooded with messages from people struggling to get interview calls. This year, it’s all about facing multiple rejections.Now that many companies are hiring again and in large numbers, if you’re still struggling to land opportunities at top firms, there are three essential skills you should focus on:✅ DSA✅  System Design✅ Quality ProjectsWith the increased competition due to the rise of AI - it’s very important to not just have a basic know

In [11]:
qualified_posts = list(filter(lambda post: post["status"] == "qualified", pending_posts))

In [12]:
for post in qualified_posts:
    print(post["poster"] +  " :  "  + post["poster_bio"]) 
    print("profile" , post["profile"]);
    print("feed_text" , post["feed_text"])  if post else post
    print("href",post["href_map"])
    print("\n")

Shubham Soni :  FE-II @Amazon | DM for long term mentorship | 108K+
profile https://www.linkedin.com/in/shubham-soni-374a86175?miniProfileUrn=urn%3Ali%3Afsd_profile%3AACoAACmguogBIdbijM6YpcIganWLJ67yKyV5kd4
feed_text 
 1st Interview: Rejected2nd Interview: Rejected3rd Interview: 2nd round Rejected4th Interview: Rejected5th Interview: Cleared but Rejected due to position filled6th Interview: Failed7th Interview: HR said, "We'll let you know"8th Interview: Got the Offer ✅You're not alone in this journey. Everyone goes through this phase. The difference is, if you give up after a few failures, you’ll never see how far you can go. You just need one YES to change your life. Keep going. 🚀The key to success is not in avoiding failure but in overcoming it and continuing to try.Follow more for updates Shubham Soni ✌️𝐃𝐌 𝐟𝐨𝐫 𝐩𝐚𝐢𝐝 𝐩𝐞𝐫𝐬𝐨𝐧𝐚𝐥𝐢𝐳𝐞𝐝 𝐦𝐞𝐧𝐭𝐨𝐫𝐬𝐡𝐢𝐩. ❤️hashtag#javascript hashtag#interview hashtag#hiring hashtag#programming hashtag#reacthashtag#frontend hashtag#connections hashtag#Interview ha